# PyTorch Inference using KServe on AWS Kubeflow Distribution


In this example, we deploy a trained PyTorch CNN image classification model to predict CIFAR 10 images by running an InferenceService with TorchServe runtime which is the default installed serving runtime for PyTorch models.

We will create KServe InferenceService yaml, pipeline, and will run the inference all from this notebook 

The KServe/TorchServe integration expects following model store layout on the storage with TorchServe Model Archive and Model Configuration.


TorchServe provides a utility to package all the model artifacts into a single TorchServe Model Archive Files (MAR), after model artifacts are packaged into MAR file you then upload to the model-store under model storage path which will be S3 in this example.

In [ ]:
# Restart Kernel after you have installed the utility

# Install kserve python sdk to create and run InferenceService yaml
!pip install kserve==0.7.0

# Install torch-model-archiver, a TorchServe utility to package all the model artifacts into a single TorchServe Model Archive Files (MAR). 
!pip install torch-model-archiver

In [ ]:
import argparse
import json
import logging
import os
import sys
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision
from torchvision import datasets, transforms
import json
#import subprocess
from model import Net

from kubernetes import client 
from kserve import KServeClient
from kserve import constants
from kserve import utils
from kserve import V1beta1InferenceService
from kserve import V1beta1InferenceServiceSpec
from kserve import V1beta1PredictorSpec
from kserve import V1beta1SKLearnSpec
from kserve import V1beta1TorchServeSpec
from kserve import V1beta1TorchServeSpec
from kubernetes.client import models as k8s

import kfp
from kfp import components
from kubeflow.training.utils import utils
from kfp import dsl
from kfp import compiler

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

In [ ]:
# Load the model created by training step  
model = Net()

## Load the model 

In [ ]:
# Same bucket name that we used in notebook 1_submit_pytorchdist_k8s.ipynb
s3_bucket_name='kserve-model-20230527042622'
model_name='model-kserve.pth'
!aws s3 cp s3://{s3_bucket_name}/{model_name} ./kserve_inference

## Create model store required by KServe/TorchServe integration

In [ ]:
# Generate the MAR file with torch-model-archiver
!$HOME/.local/bin/torch-model-archiver -f --model-name cifar --version 1.0 --model-file ./kserve_inference/model.py --serialized-file ./kserve_inference/model-kserve.pth --handler  ./kserve_inference/model_handler.py && mv cifar.mar ./kserve_inference/

In [ ]:
#Clean up the madel_layout package if it is alread there
#!rm -r model_layout

In [ ]:
#Create a model store layout required by the PyTorch Serve
!mkdir -p kserve_inference/model_layout/config 
!mkdir -p kserve_inference/model_layout/model-store

In [ ]:
# Copy the model archive
!cp kserve_inference/cifar.mar kserve_inference/model_layout/model-store/
!cp kserve_inference/config.properties kserve_inference/model_layout/config/

In [ ]:
# Load the model package to S3 bucket 
!aws s3 cp ./kserve_inference/model_layout s3://{s3_bucket_name}/ --recursive

In [ ]:
namespace = utils.get_default_target_namespace()
namespace

## Create InferenceService yaml

In [ ]:
resource_req=k8s.V1ResourceRequirements(requests={'cpu':'100m'})
predictor_spec=V1beta1PredictorSpec(service_account_name="aws-sa",
                                    pytorch=(V1beta1TorchServeSpec(
                                   resources=resource_req,
                                   storage_uri=f's3://{s3_bucket_name}')))
    
#predictor_spec

In [ ]:
inference_endpoint='image-classify'
kserve_version='v1beta1'
api_version = constants.KSERVE_GROUP + '/' + kserve_version

isvc = V1beta1InferenceService(api_version=api_version,
                               kind=constants.KSERVE_KIND,
                               metadata=client.V1ObjectMeta(
                                   name=inference_endpoint, namespace=namespace, annotations={'sidecar.istio.io/inject':'false'}),
                               spec=V1beta1InferenceServiceSpec(
                               predictor=predictor_spec)
)

#isvc

In [ ]:
#Create KServe Inference EndPoint
KServe = KServeClient()
KServe.create(isvc)

In [ ]:
#Delete the KServe Inference EndPoint 
#name="image-classify"
#KServe.delete(name, namespace=namespace)

In [ ]:
#KServe.get(inference_endpoint, namespace=namespace)

In [ ]:
KServe.get(inference_endpoint, namespace=namespace, timeout_seconds=120)